In [8]:
import polars as pl
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error
from functions import read_dataframe


## Q1. Downloading the data
We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

In [2]:
yellow_tripdata_2023_01 = pl.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet")
yellow_tripdata_2023_02 = pl.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet")

print(f"There are {len(yellow_tripdata_2023_01.columns)} columns in the January data.")

There are 19 columns in the January data.


## Q2. Computing duration
Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

In [3]:
january_data_yellow = (
        yellow_tripdata_2023_01
        .with_columns(
            (pl.col("tpep_dropoff_datetime") - pl.col("tpep_pickup_datetime"))
            .dt.total_seconds()
            .alias("duration")
        )
        .with_columns(
            (pl.col("duration") / 60)
            .alias("duration_minutes")
        ))

std_dev_duration_january = january_data_yellow["duration_minutes"].std()

print(f"The standard deviation of the trips duration in January is {std_dev_duration_january:.2f}.")


The standard deviation of the trips duration in January is 42.59.


## Q3. Dropping outliers
Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

In [4]:
january_data_yellow_filtered = (
    january_data_yellow
    .filter((pl.col("duration_minutes") > 1) & (pl.col("duration_minutes") <= 60))
    )

print(f"The fraction of the records left after dropping the outliers is {len(january_data_yellow_filtered) / len(january_data_yellow):.2f}.")


The fraction of the records left after dropping the outliers is 0.98.


## Q4. One-hot encoding
Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

- Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
- Fit a dictionary vectorizer
- Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

In [5]:
january_data_yellow_filtered = (
    january_data_yellow_filtered
    .with_columns(
        pl.col(["PULocationID", "DOLocationID"]).cast(pl.Utf8)
    )
)

dv = DictVectorizer()
X_train = dv.fit_transform(january_data_yellow_filtered.select(["PULocationID", "DOLocationID"]).to_dicts())

print(f"The dimensionality of the feature matrix is {X_train.shape[1]}.")

The dimensionality of the feature matrix is 515.


## Q5. Training a model
Now let's use the feature matrix from the previous step to train a model.

- Train a plain linear regression model with default parameters, where duration is the response variable
- Calculate the RMSE of the model on the training data

What's the RMSE on train?

In [7]:
y_train = january_data_yellow_filtered["duration_minutes"]

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_train)

rmse = root_mean_squared_error(y_train, y_pred)
print(f"The RMSE on train is {rmse:.2f}.")

The RMSE on train is 7.65.


## Q6. Evaluating the model
Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

In [14]:
# preparing the validation data
february_data_yellow, df_val, categorical_features = read_dataframe(yellow_tripdata_2023_02, 
                                                                    pu_col = "tpep_pickup_datetime", 
                                                                    do_col = "tpep_dropoff_datetime")

X_val = dv.transform(df_val.select(["PULocationID", "DOLocationID", "trip_distance"]).to_dicts())
y_val = df_val["duration_minutes"]

# make predictions
y_pred = model.predict(X_val)

# calculate the RMSE
rmse = root_mean_squared_error(y_val, y_pred)
print(f"The RMSE on validation is {rmse:.2f}.")


The RMSE on validation is 7.81.
